# NEWEST 

In [104]:
# IW CODE: https://github.com/sarahmbock/COS-IW-BOCK/blob/main/DANN%20CODE.ipynb

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import time
import os
import torch.nn as nn
from torch.autograd import Function
import torchvision
import torch.utils.data as data
from PIL import Image
import os
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np 
import torch.nn.functional as F
import torch.nn.init as init
import pandas as pd
import sklearn.preprocessing as preprocessing

In [2]:
# Gradient reversal layer
class GRL(Function):
    @staticmethod
    def forward(ctx, x, constant):
        ctx.constant = constant
        return x.view_as(x) * constant
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.constant, None

In [3]:
import torch.nn.functional as F
from torch.autograd import Function

class GradReverse(Function):
    
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)
    
    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.lambda_
        return output, None

def grad_reverse(x, lambda_=1.0):
    return GradReverse.apply(x, lambda_)


In [4]:
# Can adjust NN Sizes
class Dann(nn.Module):
    def __init__(self):
        super(Dann, self).__init__()
        self.f = nn.Sequential(
                nn.Linear(85, 64),
            nn.ReLU(),
        )
        # Label classifier
        self.lc = nn.Sequential(
    nn.Linear(64, 32),
    nn.Sigmoid(),
    nn.Linear(32, 32),
    nn.Sigmoid(),
    nn.Linear(32, 2),
    nn.Sigmoid()
)
        
        # Domain classifier
        self.dc = nn.Sequential(nn.Linear(64, 36),
    nn.Sigmoid(),
    nn.Linear(36, 36), nn.Sigmoid(),              
    nn.Linear(36, 2),
    nn.Sigmoid()
                                
)
    def forward(self, x,alpha):
        x = self.f(x)
        x = x.view(-1, 64) # must be same dimension as output size from init()
        y = GRL.apply(x, alpha)
        x = self.lc(x)
        y = self.dc(y)
        x=x.view(x.shape[0],-1)
        y=y.view(y.shape[0],-1)
        return x, y

# DATA

In [5]:
# Adult data code
# Change these to local file if available
train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'

features = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"] 


# This will download 3.8M
original_train = pd.read_csv(train_url, names=features, sep=r'\s*,\s*', 
                             engine='python', na_values="?")
# This will download 1.9M
original_test = pd.read_csv(test_url, names=features, sep=r'\s*,\s*', 
                            engine='python', na_values="?", skiprows=1)



num_train = len(original_train)
original = pd.concat([original_train, original_test])
roc_original = original
labels = original['Target']
labels = labels.replace('<=50K', 0).replace('>50K', 1)
labels = labels.replace('<=50K.', 0).replace('>50K.', 1)


# Preprocess
# Redundant column
del original["Education"]
# Remove target variable
del original["Target"]


def data_transform(df):
    """Normalize features."""
    binary_data = pd.get_dummies(df)
    feature_cols = binary_data[binary_data.columns[:-2]]
    scaler = preprocessing.StandardScaler()
    data = pd.DataFrame(scaler.fit_transform(feature_cols), columns=feature_cols.columns)
    return data

data = data_transform(original)

In [6]:
# Adult code, seperate sensitive and non sensitive
# get array of sensitive characteristics
sensitive = original.Sex.map({'Male': 0, 'Female': 1})
y = np.array(labels).flatten()
data = data.drop(['Sex_Male', 'Sex_Female'], axis=1) # Drop sensitive info from data
data_np = np.array(data)
# split to source (non-sensitive) and target (sensitive)
sensitive_idx = np.array(np.where(sensitive==1))[0].flatten()
nonsensitive_idx = np.array(np.where(sensitive!=1))[0].flatten()
data_sensitive = data_np[sensitive_idx,:]
data_nonsensitive = data_np[nonsensitive_idx,:]
y_sensitive = y[sensitive_idx]
y_nonsensitive = y[nonsensitive_idx]
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

X_tensor_source =  torch.from_numpy(data_nonsensitive).type(torch.float32)
X_tensor_target =  torch.from_numpy(data_sensitive).type(torch.float32)
y_tensor_source =torch.tensor(y_nonsensitive).type(torch.LongTensor)
y_tensor_target =torch.tensor(y_sensitive).type(torch.LongTensor)


# Split the source (white) dataset into training and test sets
X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_tensor_source, y_tensor_source, test_size=0.2, random_state=42)


# Split the target (black) into training and test sets
X_train_target, X_test_target, y_train_target, y_test_target = train_test_split(X_tensor_target, y_tensor_target, test_size=0.2, random_state=42)

# Create TensorDatasets from the training and test sets
target_train = TensorDataset(X_train_target, y_train_target)
target_test = TensorDataset(X_test_target, y_test_target)

source_train = TensorDataset(X_train_source, y_train_source)
source_test = TensorDataset(X_test_source, y_test_source)

# WEIGHTS
num_samples = len(target_train) # number of black samples
num_positives = sum(target_train[i][1] for i in range(num_samples)) # Number black with positive label
num_negatives = num_samples - num_positives
class_sample_count = np.array([num_negatives, num_positives]) 


weight = 1/class_sample_count
print('positive weight (under represented):', weight[1])
print('negative weight (over represented):', weight[0])

samples_weight = np.array([weight[t] for t in y_train_target])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))




# Create DataLoaders from the datasets
batch_size = 64
target_train = DataLoader(target_train, batch_size=batch_size, sampler=sampler) # remove sampler
target_test= DataLoader(target_test, batch_size=len(target_test), shuffle=False)



num_samples = len(source_train)
num_positives = sum(source_train[i][1] for i in range(num_samples))
num_negatives = num_samples - num_positives
class_sample_count = np.array([num_negatives, num_positives])

weight = 1/class_sample_count
print('positive weight (under represented):', weight[1])
print('negative weight (over represented):', weight[0])

samples_weight = np.array([weight[t] for t in y_train_source])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))


source_train = DataLoader(source_train, batch_size=batch_size, sampler=sampler) # remove sampler
source_test = DataLoader(source_test, batch_size=len(source_test), shuffle=False)

combined_tensor = torch.cat((X_train_source, X_train_target), dim=0)
# Weight with respect to sensitive feature itself
y_domain = torch.zeros(len(combined_tensor), dtype=torch.long)
y_domain[len(X_train_source):] = 1
print(sum(y_domain).item())
print(y_domain)
combined_set =  TensorDataset(combined_tensor, y_domain)
class_sample_count = np.array([len(X_train_source), len(X_train_target)])
print(class_sample_count)
weight = 1/class_sample_count
print(weight)
samples_weight = np.array([weight[t] for t in y_domain])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

combined_loader =  DataLoader(combined_set, batch_size=batch_size) # Removed sensitive characteristic data loader



# Combine white and black to train credit predictor
y_label_combined = torch.cat((y_train_source,y_train_target), dim=0)
X_label_combined = torch.cat((X_train_source,X_train_target), dim=0)
num_samples = len(X_label_combined)
num_positives = sum(y_label_combined[i] for i in range(num_samples)) # Number of positives for label predictor
num_negatives = num_samples - num_positives
print(num_positives.item())
print(num_negatives.item())
class_sample_count = np.array([num_negatives.item(), num_positives.item()])
weight = 1/class_sample_count
print(weight)
samples_weight = np.array([weight[t] for t in y_label_combined])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))
combined_label_set = TensorDataset(X_label_combined, y_label_combined)
combined_label_loader = DataLoader(combined_label_set, batch_size=batch_size) # Remove sampler


Index(['Age', 'fnlwgt', 'Education-Num', 'Capital Gain', 'Capital Loss',
       'Hours per week', 'Workclass_Federal-gov', 'Workclass_Local-gov',
       'Workclass_Never-worked', 'Workclass_Private', 'Workclass_Self-emp-inc',
       'Workclass_Self-emp-not-inc', 'Workclass_State-gov',
       'Workclass_Without-pay', 'Martial Status_Divorced',
       'Martial Status_Married-AF-spouse', 'Martial Status_Married-civ-spouse',
       'Martial Status_Married-spouse-absent', 'Martial Status_Never-married',
       'Martial Status_Separated', 'Martial Status_Widowed',
       'Occupation_Adm-clerical', 'Occupation_Armed-Forces',
       'Occupation_Craft-repair', 'Occupation_Exec-managerial',
       'Occupation_Farming-fishing', 'Occupation_Handlers-cleaners',
       'Occupation_Machine-op-inspct', 'Occupation_Other-service',
       'Occupation_Priv-house-serv', 'Occupation_Prof-specialty',
       'Occupation_Protective-serv', 'Occupation_Sales',
       'Occupation_Tech-support', 'Occupation_Trans

In [10]:
def test(model, source_train_loader, source_test_loader, target_test_loader, lambda_):
    # Set model to evaluation mode
    model.eval()
    model.lc.eval()
    model.dc.eval()
    model.f.eval()

    # Define variables for tracking accuracy
    source_label_correct = 0
    source_domain_correct = 0
    target_domain_correct = 0

    # Evaluate on source train set
    for i, (data, labels) in enumerate(source_train_loader):
        # Set domain label to 0 (i.e., source domain)
        domain_labels = torch.zeros(data.shape[0]).long()

        # Forward pass through the model
        label_preds, domain_preds = model(data, alpha=lambda_)
        #print(domain_preds)
        domain_preds = model.dc(grad_reverse(model.f(data), lambda_))
        

        # Calculate accuracy of label classifier
        _, label_preds = torch.max(label_preds, 1)
        source_label_correct += (label_preds == labels).sum().item()

        # Calculate accuracy of domain classifier
        _, domain_preds = torch.max(domain_preds, 1)
        source_domain_correct += (domain_preds == domain_labels).sum().item()
    #num_ones_predicted =0
    # Evaluate on source test set
    for i, (data, labels) in enumerate(source_test_loader):
        # Set domain label to 0 (i.e., source domain)
        domain_labels = torch.zeros(data.shape[0]).long()

        # Forward pass through the model
        label_preds, domain_preds = model(data, alpha=lambda_)
        domain_preds = model.dc(grad_reverse(model.f(data), lambda_))

        # Calculate accuracy of label classifier
        _, label_preds = torch.max(label_preds, 1)
        source_label_correct += (label_preds == labels).sum().item()
        #num_ones_predicted += list(label_preds.numpy()).count(1)

        # Calculate accuracy of domain classifier
        _, domain_preds = torch.max(domain_preds, 1)
        source_domain_correct += (domain_preds == domain_labels).sum().item()
    
    # Evaluate on target test set
    target_label_correct =0
    for i, (data, labels) in enumerate(target_test_loader):
        # Set domain label to 1 (i.e., target domain)
        domain_labels = torch.ones(data.shape[0]).long()

        # Forward pass through the model
        label_preds, domain_preds = model(data, alpha=lambda_)
        domain_preds = model.dc(grad_reverse(model.f(data), lambda_))
        
        # Calculate accuracy of label classifier
        _, label_preds = torch.max(label_preds, 1)
        target_label_correct += (label_preds == labels).sum().item()
        #num_ones_predicted += list(label_preds.numpy()).count(1)
        
        # Calculate accuracy of domain classifier
        _, domain_preds = torch.max(domain_preds, 1)
        target_domain_correct += (domain_preds == domain_labels).sum().item()
        

    # Calculate and print accuracy
    source_label_accuracy = 100. * source_label_correct / (len(source_train_loader.dataset) + len(source_test_loader.dataset))
    source_domain_accuracy = 100. * source_domain_correct / (len(source_train_loader.dataset) + len(source_test_loader.dataset))
    target_label_accuracy = 100. * target_label_correct / (len(target_test_loader.dataset))
    target_domain_accuracy = 100. * target_domain_correct / len(target_test_loader.dataset)
    print('Source Label Accuracy: {:.2f}%'.format(source_label_accuracy))
    print('Target Label Accuracy: {:.2f}%'.format(target_label_accuracy))
    print('Source Domain Accuracy: {:.2f}%'.format(source_domain_accuracy))
    print('Target Domain Accuracy: {:.2f}%'.format(target_domain_accuracy))
    #print('Number of loan defaults predicted: ', num_ones_predicted / (len(target_test_loader.dataset)+ len(source_test_loader.dataset)))


In [11]:
from sklearn.metrics import confusion_matrix
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score
def test_fairness(model, lambda_):
    with torch.no_grad():
        predicted_labels_white = []
        true_labels_white = []
        predicted_domain = []
        source_label_correct =0
        source_domain_correct =0
        for i, (data, labels) in enumerate(source_test):
            # Set domain label to 0 (i.e., source domain)
            domain_labels = torch.zeros(data.shape[0]).long()

            # Forward pass through the model
            label_preds, domain_preds = model(data, alpha=lambda_)
            domain_preds = model.dc(grad_reverse(model.f(data), lambda_=lambda_))

            # Calculate accuracy of label classifier
            _, label_preds = torch.max(label_preds, 1)
            source_label_correct += (label_preds == labels).sum().item()
            #num_ones_predicted += list(label_preds.numpy()).count(1)

            # Calculate accuracy of domain classifier
            _, domain_preds = torch.max(domain_preds, 1)
            source_domain_correct += (domain_preds == domain_labels).sum().item()
            predicted_labels_white += label_preds
            true_labels_white += labels
            predicted_domain += domain_preds
        conf_matrix = confusion_matrix(true_labels_white, predicted_labels_white)
        print('Credit Confusion Matrix White: ')
        print(conf_matrix)
        fpr = conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0]) 
        print('False positive white: ', fpr)
        
        predicted_labels = []
        true_labels = []
        predicted_domain = []
        target_label_correct =0
        target_domain_correct =0
        for i, (data, labels) in enumerate(target_test):
            # Set domain label to 0 (i.e., source domain)
            domain_labels = torch.ones(data.shape[0]).long()

            # Forward pass through the model
            label_preds, domain_preds = model(data, alpha=lambda_)
            domain_preds = model.dc(grad_reverse(model.f(data), lambda_=lambda_))

            # Calculate accuracy of label classifier
            _, label_preds = torch.max(label_preds, 1)
            target_label_correct += (label_preds == labels).sum().item()
            #num_ones_predicted += list(label_preds.numpy()).count(1)

            # Calculate accuracy of domain classifier
            _, domain_preds = torch.max(domain_preds, 1)
            target_domain_correct += (domain_preds == domain_labels).sum().item()
            predicted_labels += label_preds
            true_labels += labels
            predicted_domain += domain_preds
        conf_matrix = confusion_matrix(true_labels, predicted_labels)
        print('Credit Confusion Matrix Black: ')
        print(conf_matrix)
        fpr = conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0]) 
        print('False positive black: ', fpr)
        
        # predictive Parity
        metric_frame = MetricFrame(metrics=accuracy_score, y_true=true_labels_white + true_labels,
                           y_pred=predicted_labels_white + predicted_labels,
                           sensitive_features=[1]*len(true_labels_white) + [0]*len(true_labels)) #are white and black mixed up here?

        predictive_parity = metric_frame.difference(method='between_groups')

        print("Predictive parity: ", predictive_parity)
        
        
        
    

In [12]:
def test_lc(model, lambda_):
    model.eval()
    model.lc.eval()
    model.f.eval()
    correct = 0
    ones = 0
    for data, labels in source_test:
        embedding = model.f(data)
        label_preds = model.lc(embedding)
        print(label_preds)
        label_preds = torch.max(label_preds, 1)
        print(label_preds)
        # Get number of correct predictions
        source_label_correct += (label_preds == labels).sum().item()
        ones += list(label_preds.numpy()).count(1)
        
        # get number of ones
    source_test_label_acc = source_label_correct / len(source_test_loader.dataset)
    print('Accuracy of label classifier on source test: ', source_test_label_acc )
    print('Late loans predicted on source test: ', ones/ len(source_test_loader.dataset))
        
        

In [13]:
def test_dc(model, lambda_):
    model.eval()
    model.lc.eval()
    model.f.eval()
    model.dc.eval()
    correct = 0
    black = 0
    actually_black = 0
    for data, labels in combined_loader:
        combined_data, domain_labels = data, labels
        embedded = grad_reverse(model.f(combined_data), lambda_=lambda_)
        domain_preds = model.dc(embedded)
        #print(domain_preds)
        _, domain_preds = torch.max(domain_preds, 1)
        #domain_loss = domain_criterion(domain_preds, domain_labels)
        #print(domain_preds)
        correct += (domain_preds == labels).sum().item()
        black += list(domain_preds.numpy()).count(1)
        actually_black += list(labels.numpy()).count(1)
    accuracy = correct / len(combined_loader.dataset)
    print('domain accuracy on train data: ', accuracy)
    print('black predicted: ', black/(len(combined_loader.dataset)))
    print('num actually black: ', actually_black/(len(combined_loader.dataset)))
    black = 0
    correct = 0
    actually_black = 0
    for data, labels in source_test:
        embedded = grad_reverse(model.f(data), lambda_=lambda_)
        domain_preds = model.dc(embedded)
        _, domain_preds = torch.max(domain_preds, 1)
        labels = torch.zeros(data.shape[0]).long()
        correct += (domain_preds == labels).sum().item()
        black += list(domain_preds.numpy()).count(1)
    for data, labels in target_test:
        embedded = grad_reverse(model.f(data), lambda_=lambda_)
        domain_preds = model.dc(embedded)
        _, domain_preds = torch.max(domain_preds, 1)
        labels = torch.ones(data.shape[0]).long()
        correct += (domain_preds == labels).sum().item()
        black += list(domain_preds.numpy()).count(1)
    test_accuracy = correct / (len(target_test.dataset)+ len(source_test.dataset))
    print('domain test accuracy: ', test_accuracy)
    print('black pred test: ', black/(len(target_test.dataset)+ len(source_test.dataset)))
    print('actual black = ', len(target_test.dataset)/(len(target_test.dataset)+ len(source_test.dataset)))
          
        
        

        
    
    
    

In [14]:
def train(dann, num_epochs, feature_optimizer, label_epochs, domain_epochs, source_loader, target_loader, label_criterion, domain_criterion,
         label_optimizer, domain_optimizer, lambda_):
    for epoch in range(num_epochs):
    # Set the DANN model to training mode
        dann.train()
        # Set the number of batches for each epoch

            
        # Train the label classifier for label_epochs epochs
        for j in range(label_epochs):
            dann.lc.train()
            for data, labels in combined_label_loader:
                source_data, source_labels = data, labels
                label_optimizer.zero_grad()
                feature_embed = dann.f(source_data)
                label_preds = dann.lc(feature_embed)
                label_loss = label_criterion(label_preds, source_labels)
                label_loss.backward()
                label_optimizer.step()
        print('label trained')
        dann.lc.eval()

        # Train the domain classifier for domain_epochs epochs
        for j in range(domain_epochs):
            dann.dc.train()
            for data, labels in combined_loader:
                combined_data, domain_labels = data, labels
                domain_optimizer.zero_grad()
                embedded = grad_reverse(dann.f(combined_data), lambda_=lambda_)
                domain_preds = dann.dc(embedded)
                domain_loss = domain_criterion(domain_preds, domain_labels)
                domain_loss.backward()
                domain_optimizer.step()
        print('domain trained')
        dann.dc.eval()


        dann.f.train()
        dann.dc.train()
        dann.lc.train()
        dann.train()
        
        for data, labels in source_loader:
            feature_optimizer.zero_grad()
            #dann.zero_grad()
            #feature_embed_label, feature_embed_domain = dann(data, lambda_)
            #feature_embed_domain = grad_reverse(dann.f(data), lambda_=lambda_)
            #label_preds = dann.lc(feature_embed_label)
            #domain_preds = dann.dc(feature_embed_domain)
            label_preds, domain_preds = dann(data, lambda_)
            label_loss = label_criterion(label_preds, labels)
            domain_labels = torch.zeros(len(data), dtype=torch.long)
            domain_loss = domain_criterion(domain_preds, domain_labels)
            total_loss = label_loss + (domain_loss* lambda_)
            total_loss.backward()
            feature_optimizer.step()
            
        for data, labels in target_loader:
            feature_optimizer.zero_grad()
            #dann.zero_grad()
            #feature_embed = dann.f(data)
            #label_preds = dann.lc(feature_embed)
            #feature_embed_domain = grad_reverse(dann.f(data), lambda_=lambda_)
            #domain_preds=dann.dc(feature_embed_domain)
            #print(domain_preds)
            label_preds, domain_preds = dann(data, lambda_)
            label_loss = label_criterion(label_preds, labels)
            domain_labels = torch.ones(len(data), dtype=torch.long)
            domain_loss = domain_criterion(domain_preds, domain_labels)
            total_loss = label_loss + (domain_loss*lambda_)
            total_loss.backward()
            feature_optimizer.step()
        print('feature extractor trained')
        dann.f.eval()
        dann.dc.eval()
        dann.lc.eval()
        dann.eval()
        print('Epoch [{}/{}], Label Loss: {:.4f}, Domain Loss: {:.4f}'.format(epoch+1, num_epochs, label_loss.item(), domain_loss.item()))
        test(dann, source_train, source_test, target_test, lambda_)
        #test_dc(dann, lambda_)
        #test_fairness(dann, lambda_)
        print()

# Print the loss for this epoch
    
    


In [15]:
def train_for_lambda(lambda_):
    model = Dann()

    # initialize weights
    for m in model.f.modules():
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
    for m in model.lc.modules():
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
    for m in model.dc.modules():
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)

    # Define optimizer for the feature extractor and label classifier
    optimizer_lc = torch.optim.Adam(list(model.f.parameters()) + list(model.lc.parameters()), lr=0.00001, weight_decay=1e-3)
    criterion_lc = nn.CrossEntropyLoss()

    # Define optimizer for the domain classifier
    optimizer_dc = torch.optim.Adam(model.dc.parameters(), lr=0.0001, weight_decay=1e-3)
    criterion_dc = nn.CrossEntropyLoss()

    f_optimizer = torch.optim.Adam(model.f.parameters(), lr=0.0001, weight_decay=1e-3)
    # Define number of epochs to train the label and domain classifiers
    n_lc_epochs = 10
    n_dc_epochs = 10

    _lambda = lambda_
    lambda_ = _lambda
    # Training loop
    num_epochs = 20
    train(model, num_epochs, optimizer_lc, 20, domain_epochs =30, 
          source_loader=source_train, target_loader= target_train, 
          label_criterion =criterion_lc, domain_criterion=criterion_dc,
         label_optimizer = optimizer_lc, domain_optimizer = optimizer_dc, lambda_ = lambda_)
    return model



In [122]:
'''lambda = 0
label trained
domain trained
feature extractor trained
Epoch [30/30], Label Loss: 0.3613, Domain Loss: 0.3133
Source Label Accuracy: 96.33%
Target Label Accuracy: 97.27%
Source Domain Accuracy: 89.32%
Target Domain Accuracy: 84.73%
domain accuracy on train data:  0.901286677105317
black predicted:  0.4585063653073729
num actually black:  0.48519300156579753
domain test accuracy:  0.906071331336782
black pred test:  0.19793084671930303
actual black =  0.14974135583991288
Credit Confusion Matrix White: 
[[2673   64]
 [  40  346]]
False positive white:  0.023383266350018268
Credit Confusion Matrix Black: 
[[376   7]
 [  8 159]]
False positive black:  0.018276762402088774

'''

'lambda = 0\nlabel trained\ndomain trained\nfeature extractor trained\nEpoch [30/30], Label Loss: 0.3613, Domain Loss: 0.3133\nSource Label Accuracy: 96.33%\nTarget Label Accuracy: 97.27%\nSource Domain Accuracy: 89.32%\nTarget Domain Accuracy: 84.73%\ndomain accuracy on train data:  0.901286677105317\nblack predicted:  0.4585063653073729\nnum actually black:  0.48519300156579753\ndomain test accuracy:  0.906071331336782\nblack pred test:  0.19793084671930303\nactual black =  0.14974135583991288\nCredit Confusion Matrix White: \n[[2673   64]\n [  40  346]]\nFalse positive white:  0.023383266350018268\nCredit Confusion Matrix Black: \n[[376   7]\n [  8 159]]\nFalse positive black:  0.018276762402088774\n\n'

In [16]:
def write_results(lam, model, test_src, test_targ):
    file_name = 'results_mult_adult'+str(lam)+'.txt'
    my_str = 'Lambda: ' +str(lam) + '\n\n'
    results = get_results(model, test_src, test_targ, lam)
    # test accuracy-- source, target, and overall
    my_str += 'Label Classifier Accuracy (Test Data) \n'
    my_str += 'Source (White): ' + str(results['source_label_accuracy']) + '\n'
    my_str += 'Target (Black): ' + str(results['target_label_accuracy'])+ '\n'
    my_str += 'Overall: ' + str(results['overall_label_accuracy'])+ '\n\n'
    # domain test accuracy-- source, target, and overall
    my_str += 'Domain Classifier Accuracy (Test Data) \n'
    my_str += 'Source (White): ' + str(results['source_domain_accuracy']) + '\n'
    my_str += 'Target (Black): ' + str(results['target_domain_accuracy'])+ '\n'
    my_str += 'Overall: ' + str(results['overall_domain_accuracy'])+ '\n\n'
    # false positives-- source, target, and overall
    my_str += 'False Positive Rate (Test Data) \n'
    my_str += 'Source (White): ' + str(results['fpr_white']) + '\n'
    my_str += 'Target (Black): ' + str(results['fpr_black'])+ '\n'
    my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'
    my_str += 'Overall: ' + str(results['fpr_overall'])+ '\n\n'
    
    print(my_str)
    with open(file_name, "w") as file:
        file.write(my_str)
    
    # parity
    

In [17]:
def get_results(model, test_src, test_targ, lam):
    #print('get results model')
    #print(model)
    with torch.no_grad():
        # Label predictor metrics
        source_correct = 0
        source_label_predictions = []
        source_label_true = []
        
        # Domain predictor metrics
        source_dom_correct = 0
        source_dom_predictions = []
        source_dom_true = []
        for data, labels in test_src:
            # Make label predictions
            embedded = model.f(data)
            label_preds = model.lc(embedded)
            _, label_preds = torch.max(label_preds, 1)
            source_correct += (label_preds == labels).sum().item()
            source_label_predictions += label_preds
            source_label_true+= labels
            
            # Make domain predictions
            domain_embedded = grad_reverse(model.f(data), lam)
            domain_preds = model.dc(domain_embedded)
            _, domain_preds = torch.max(domain_preds, 1)
            domain_true = torch.zeros(data.shape[0]).long()
            source_dom_correct += (domain_preds == domain_true).sum().item()
            source_dom_predictions += domain_preds
            source_dom_true += domain_true
        source_label_accuracy = 100.* source_correct / len(test_src.dataset)
        source_domain_accuracy = 100 * source_dom_correct / len(test_src.dataset)
        source_label_positive_rate = (sum(source_label_predictions)/len(source_label_predictions)).item()

        
        
        
        # FOR TARGET DATA
        # Label predictor metrics
        target_correct = 0
        target_label_predictions = []
        target_label_true = []
        
        # Domain predictor metrics
        target_dom_correct = 0
        target_dom_predictions = []
        target_dom_true = []
        for data, labels in test_targ:
            # Make label predictions
            embedded = model.f(data)
            label_preds = model.lc(embedded)
            _, label_preds = torch.max(label_preds, 1)
            target_correct += (label_preds == labels).sum().item()
            target_label_predictions += label_preds
            target_label_true+= labels
            
            # Make domain predictions
            domain_embedded = grad_reverse(model.f(data), lam)
            domain_preds = model.dc(domain_embedded)
            _, domain_preds = torch.max(domain_preds, 1)
            domain_true = torch.ones(data.shape[0]).long()
            target_dom_correct += (domain_preds == domain_true).sum().item()
            target_dom_predictions += domain_preds
            target_dom_true += domain_true
        target_label_accuracy = 100.* target_correct / len(test_targ.dataset)
        target_domain_accuracy = 100 * target_dom_correct / len(test_targ.dataset)
        target_label_positive_rate = (sum(target_label_predictions)/len(target_label_predictions)).item()

        
        # OVERALL
        overall_label_accuracy = 100.*(target_correct + source_correct)/(len(test_src.dataset)+len(test_targ.dataset))
        overall_domain_accuracy = 100.*(target_dom_correct + source_dom_correct)/(len(test_src.dataset)+len(test_targ.dataset))
        overall_label_positive_rate = ((sum(source_label_predictions) + sum(target_label_predictions))/(len(target_label_predictions)+len(source_label_predictions))).item()

        
        # False positive rates
        conf_matrix = confusion_matrix(source_label_true, source_label_predictions)
        fpr_white = 100.* conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0])
        
        conf_matrix = confusion_matrix(target_label_true, target_label_predictions)
        fpr_black = 100.* conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0])
        
        conf_matrix = confusion_matrix(target_label_true+source_label_true, target_label_predictions+source_label_predictions)
        fpr_overall = 100. *conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0])
        
        
        
        
        
        
        
        
        results = {'source_label_accuracy': source_label_accuracy, 'source_domain_accuracy':source_domain_accuracy,
                  'target_label_accuracy': target_label_accuracy, 'target_domain_accuracy':target_domain_accuracy,
                   'overall_label_accuracy': overall_label_accuracy, 'overall_domain_accuracy': overall_domain_accuracy,
                   'fpr_white': fpr_white, 'fpr_black': fpr_black, 'fpr_overall': fpr_overall,
                     'source_label_positive_rate': source_label_positive_rate, 'target_label_positive_rate': target_label_positive_rate,
                   'overall_label_positive_rate': overall_label_positive_rate}
        return results
        
        
            
            
            
                
            
            
    

In [ ]:
models = {}
for lam in range(10):
    lam = 0.1*lam
    model = train_for_lambda(lam)
    models[lam] = model
    write_results(lam, model, source_test, target_test)
    
    
    

In [19]:
def write_conf_results(lam, test_src, test_targ):
    file_name = 'results/results_conf_Adult'+str(lam)+'.txt'
    my_str = 'Lambda: ' +str(lam) + '\n\n'
    for i in range(10):
        my_str += 'run: ' + str(i+1)+'/15\n'
        model = train_for_lambda(lam)
        model_file_name = 'model_results/adult_original_'+str(lam)+'_'+str(i)+'.pt'
        torch.save(model.state_dict(), model_file_name)
        results = get_results(model, test_src, test_targ, lam)
        # test accuracy-- source, target, and overall
        my_str += 'Label Classifier Accuracy (Test Data) \n'
        my_str += 'Source (White): ' + str(results['source_label_accuracy']) + '\n'
        my_str += 'Target (Black): ' + str(results['target_label_accuracy'])+ '\n'
        my_str += 'Overall: ' + str(results['overall_label_accuracy'])+ '\n\n'
        # domain test accuracy-- source, target, and overall
        my_str += 'Domain Classifier Accuracy (Test Data) \n'
        my_str += 'Source (White): ' + str(results['source_domain_accuracy']) + '\n'
        my_str += 'Target (Black): ' + str(results['target_domain_accuracy'])+ '\n'
        my_str += 'Overall: ' + str(results['overall_domain_accuracy'])+ '\n\n'
        # false positives-- source, target, and overall
        my_str += 'False Positive Rate (Test Data) \n'
        my_str += 'Source (White): ' + str(results['fpr_white']) + '\n'
        my_str += 'Target (Black): ' + str(results['fpr_black'])+ '\n'
        my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'
        my_str += 'Overall: ' + str(results['fpr_overall'])+ '\n\n'
        my_str += 'Source Label Positive Rate: ' + str(results['source_label_positive_rate']) + '\n'
        my_str += 'Target Label Positive Rate: ' + str(results['target_label_positive_rate']) + '\n'
        my_str += 'Overall Label Positive Rate: ' + str(results['overall_label_positive_rate']) + '\n\n'
        print(my_str)

    print(my_str)
    with open(file_name, "w") as file:
        file.write(my_str)

In [20]:
for l in range(10):
    l = l * .1
    write_conf_results(l, source_test, target_test)

label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.7877, Domain Loss: 0.9752
Source Label Accuracy: 53.95%
Target Label Accuracy: 88.95%
Source Domain Accuracy: 73.96%
Target Domain Accuracy: 73.91%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.6888, Domain Loss: 0.9432
Source Label Accuracy: 69.58%
Target Label Accuracy: 91.08%
Source Domain Accuracy: 70.60%
Target Domain Accuracy: 82.90%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.6085, Domain Loss: 0.8967
Source Label Accuracy: 77.75%
Target Label Accuracy: 91.94%
Source Domain Accuracy: 66.71%
Target Domain Accuracy: 97.72%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.5546, Domain Loss: 0.8764
Source Label Accuracy: 78.47%
Target Label Accuracy: 91.76%
Source Domain Accuracy: 66.40%
Target Domain Accuracy: 98.15%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab